In [1]:
%reload_ext watermark
%watermark -v -p os,glob,cv2,shutil,joblib,gc,random,re,tensorflow,numpy,tqdm,matplotlib,pandas,sklearn,datetime,random,warning

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.34.0

os        : unknown
glob      : unknown
cv2       : 4.6.0
shutil    : unknown
joblib    : 1.1.0
gc        : unknown
random    : unknown
re        : 2.2.1
tensorflow: 2.8.0
numpy     : 1.22.3
tqdm      : 4.63.1
matplotlib: 3.5.1
pandas    : 1.4.1
sklearn   : 1.0.2
datetime  : unknown



In [2]:
import os, glob, cv2, shutil, joblib,gc, random, re
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
import sklearn
import random
import warnings
warnings.filterwarnings(action='ignore')
import datetime
print(datetime.datetime.today())

2022-11-29 17:51:33.101266


In [3]:
def video_gray2rgb(df):
    out=np.zeros(df.shape[:3]+(3,),dtype='u1')
    for i in range(df.shape[0]):
        img=cv2.cvtColor(df[i,:,:,0],cv2.COLOR_GRAY2RGB)
        out[i,:,:,:]=img   
        del img
    return out
def batch_avi(data,idx,BATCH_SIZE=1024,font=cv2.FONT_HERSHEY_SIMPLEX):
    def label_vis(data,i,font):
        input_img=cv2.cvtColor(data[i,:,:,0],cv2.COLOR_GRAY2RGB
                            ).astype('u1')
        
        pred_img=cv2.cvtColor(data[i,:,:,1]*255,cv2.COLOR_GRAY2RGB
                            ).astype('u1')
        pred_img[:,:,(0,2)]=0
        
        label_img=cv2.addWeighted(input_img,1,pred_img,.2,0)
        text=f"Index : {i}"
        cv2.putText(label_img,text, (80,30),font,1,(255,0,0),2)
        
        return np.expand_dims(label_img,axis=0)
    return np.concatenate([label_vis(data,i, font) 
                    for i in range((BATCH_SIZE*idx),
                                   (np.min([data.shape[0],(idx+1)*BATCH_SIZE])))])

def write_avi(data,file_name,fps=30):
    fourcc = cv2.VideoWriter_fourcc(*"DIVX")
    (width, height)=data.shape[1:3]
    out = cv2.VideoWriter(file_name, fourcc, fps, (height, width),isColor=True)
    for i in range(len(data)):
        out.write(data[i])
    out.release()
    
def my_confusion_matrix(y_true,y_pred,cutoff=.5):
    label_true=y_true>=.5
    pred_true=y_pred>=cutoff
    label_false= ~ label_true
    pred_false = ~ pred_true
    tp=np.sum(label_true&pred_true)
    fn=np.sum(label_true)-tp
    fp=np.sum(label_false&pred_true)
    tn=np.sum(label_false)-fp
    return tp,fn,fp,tn

def my_generator(files,shape=(240,320)):
    def img_load(file):
        img = cv2.imread(file, cv2.IMREAD_COLOR)
        img = cv2.resize(img,shape[::-1])
        return img
    
    def generator():
        for file in files:
            split_img=img_load(file)#[np.newaxis]
            # split_img=cv2.cvtColor(split_img,cv2.COLOR_GRAY2RGB)
            split_target_img=img_load(file.replace('raw','mask'))#[np.newaxis]
            split_target_img=cv2.cvtColor(split_target_img,cv2.COLOR_BGR2GRAY)
            yield split_img/255,split_target_img/255
    return generator

print(datetime.datetime.today())

2022-11-29 17:51:33.111738


# 모델링

In [4]:
files=glob.glob("/home/data/**/**/seg/*.png")
files=[i for i in files if (os.path.exists(os.path.realpath(i))|os.path.exists(i))]
model_files=np.array(sorted([i for i in files if re.compile('raw').findall(i)]))
print(datetime.datetime.today())

2022-11-29 17:51:35.834058


## 모델링 수행

In [5]:
print(datetime.datetime.today())
dfs=list();scores=list();confutions=list()
if '세그멘테이션지표.pkl' not in os.listdir():
    kf = KFold(n_splits = 3, shuffle = False)
    output_type='float32';output_shape=((240,320,3),(240,320))
    with tf.device('/device:GPU:0'):
        tf.keras.backend.clear_session()
        tf.random.set_seed(42)
        
        model=tf.keras.models.load_model(
            './DeepVOG.h5',compile=False)
        model=tf.keras.Model(model.input,tf.unstack(model.output,axis=-1)[1])
        model.compile(
          optimizer='sgd',
          loss='mse',
          metrics=[tf.keras.metrics.MeanIoU(num_classes=1)])

    tp_sum=dict();fn_sum=dict();fp_sum=dict();tn_sum=dict()
    for K, (train_idx,test_idx) in enumerate(kf.split(model_files),1):    
        for cutoff in np.arange(0,1.1,0.05):
            cond=f"K_{K}_cutoff_{cutoff:.2f}"
            tp_sum[cond]=0;fn_sum[cond]=0;fp_sum[cond]=0;tn_sum[cond]=0
        test_files=model_files[test_idx]
        test_files = sorted(test_files)

        random.seed(42)
        te_dataset=tf.data.Dataset.from_generator(
            my_generator(test_files),
            output_shapes=output_shape,
            output_types=(output_type, output_type))
        te_gen =te_dataset.batch(1).prefetch(1)
        print(f"K = {K}, test_length : {len(test_files)}")
        """
        모델 검증
        """
        score=list();score2=list()
        for i,(x,y) in tqdm(enumerate(te_gen)):
            pred=model.predict(x)[0]
            cv2.imwrite(f"seg_pred/{test_files[i].split('/')[-1].replace('raw','pred')}",(np.round(pred)*255).astype('u1'))
            try:
                tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
                    y.numpy().flatten().astype('u1'),
                    np.round(model(x).numpy().flatten()).astype('u1')).ravel()
                iou=tp/(tp+fp+fn)
                score.append(iou)
            except:
                score.append(1)
        confutions.append(score)
        scores.append(np.mean(score))
print(datetime.datetime.today())

2022-11-29 17:51:35.845337


2022-11-29 17:51:35.912294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:51:35.917019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:51:35.917339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:51:35.917865: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

K = 1, test_length : 1723


0it [00:00, ?it/s]2022-11-29 17:51:37.426189: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
1723it [03:36,  7.96it/s]


K = 2, test_length : 1723


1723it [03:37,  7.92it/s]


K = 3, test_length : 1722


1722it [03:35,  7.97it/s]

2022-11-29 18:02:26.947190


In [6]:
joblib.dump(score,"세그멘테이션지표.pkl")

['세그멘테이션지표.pkl']

In [7]:
with open("세그멘테이션지표.json", 'w') as outfile:
    json.dump(scores, outfile)

In [10]:
print(scores)
print(datetime.datetime.today())

[0.8427257999059942, 0.8418996592571334, 0.8449585498361211]
2022-11-29 18:02:57.568781
